In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pdb
import os, re
from collections import OrderedDict

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_summary import DataFrameSummary

from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as spacy_STOPWORDS
spacy_en = spacy.load('en')

from wordcloud import WordCloud, STOPWORDS

# pandas and plotting config
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', -1)

In [3]:
PATH = 'data'

os.makedirs(f'{PATH}/models', exist_ok=True)
os.makedirs(f'{PATH}/tmp', exist_ok=True)

## Examine the data

In [4]:
raw_train_df = pd.read_csv(f'{PATH}/train.csv')
test_df = pd.read_csv(f'{PATH}/test.csv')
sample_subm_df = pd.read_csv(f'{PATH}/sample_submission.csv')

Let's **look at the data**

In [5]:
display(raw_train_df.head(3))
display(test_df.head(3))
display(sample_subm_df.head(3))

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true. I'll have your account terminated.",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you did with this edit to W. S. Merwin. If you continue to do so, you will be blocked from editing. """,0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the """"points of interest"""" section you added because it seemed kind of spammy. I know you probably didn't mean to disobey the rules, but generally, a point of interest tends to be rather touristy, and quite irrelevant to an area culture. That's just my opinion, though.\n\nIf you want to reply, just put your reply here and add {{talkback|Jamiegraham08}} on my talkpage. """,0,0,0,0,0,0


,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. SS500 .jpg)==
1,6102620,"::Kentuckiana is colloquial. Even though the area is often referred to as this, it (in my opinion) has never held the encyclopedic precision of ""Louisville metropolitian area"", which has a specific U.S. Census definition. Also, apparently Kentuckiana often refers to the local television viewing area, which isn't nearly contiguous with the official metro area. As you indicate, Kentuckiana seems to be more of a slang or marketing phenomena than anything we could pin down in encyclopedic terms here. That's why we see Wikipedia language like ""the Louisville metropolitan area, sometimes referred to as Kentuckiana"". That's my take on it. — •"
2,14563293,"Hello fellow Wikipedians,\nI have just modified on [WIKI_LINK: Double Trouble (George Jones and Johnny Paycheck album)]. Please take a moment to review [EXTERNAL_LINK: my edit]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit for additional information. I made the following changes:\n*Added archive [EXTERNA_LINK: https://web.archive.org/web/20070403204132/http://www.epiccenter.com/] to [EXTERNA_LINK: http://www.epiccenter.com/]\nWhen you have finished reviewing my changes, please set the checked parameter below to true or failed to let others know (documentation at ).\nCheers.—"


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.5,0.5,0.5,0.5,0.5,0.5
1,6102620,0.5,0.5,0.5,0.5,0.5,0.5
2,14563293,0.5,0.5,0.5,0.5,0.5,0.5


## Clean

First, we'll add a "none" column, replace missing values, and get rid of newline characters

In [6]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
raw_train_df['none'] = 1 - raw_train_df[label_cols].max(axis=1)

In [7]:
raw_train_df.comment_text.fillna("<na>", inplace=True)
test_df.comment_text.fillna("<na>", inplace=True)

Lots of emojis, let's clean them up using this code:  https://www.kaggle.com/prashantkikani/pooled-gru-with-preprocessing

We'll include this list of various replacements from here (https://drive.google.com/file/d/0B1yuv8YaUVlZZ1RzMFJmc1ZsQmM/view) as well

In [8]:
repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

#https://drive.google.com/file/d/0B1yuv8YaUVlZZ1RzMFJmc1ZsQmM/view
# Aphost lookup dict
appos = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "i'd" : "I would",
    "i'd" : "I had",
    "i'll" : "I will",
    "i'm" : "I am",
    "isn't" : "is not",
    "it's" : "it is",
    "it'll":"it will",
    "i've" : "I have",
    "let's" : "let us",
    "mightn't" : "might not",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "we'd" : "we would",
    "we're" : "we are",
    "weren't" : "were not",
    "we've" : "we have",
    "what'll" : "what will",
    "what're" : "what are",
    "what's" : "what is",
    "what've" : "what have",
    "where's" : "where is",
    "who'd" : "who would",
    "who'll" : "who will",
    "who're" : "who are",
    "who's" : "who is",
    "who've" : "who have",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "'re": " are",
    "wasn't": "was not",
    "we'll":" will",
    "didn't": "did not",
    "tryin'": "trying"
}

repl = { **appos, **repl }  # repl becomes a merged dictionary with values from repl replacing those from appos

# display(repl)

Lets try this clean function from https://www.kaggle.com/jagangupta/stop-the-s-toxic-comments-eda

In [9]:
repl_keys = [i for i in repl.keys()]

def clean(comment):
    # convert to lower case , so that Hi and hi are the same
    comment = comment.lower()
    
    # remove \n 
    # torchtext cannot read the .csv files correctly if there are newline characters, so replace with " "
    comment = re.sub("\\n"," ",comment)
    
    # remove leaky elements like ip,user
    comment = re.sub("\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}"," ",comment)
    
    # removing usernames
    comment = re.sub("\[\[.*\]","",comment)
    
    # do any substitutions
    comment = " ".join([ repl[w] if (w in repl_keys) else w for w in comment.split() ])
    
    return(comment)

In [10]:
%time raw_train_df['comment_text_cleaned'] = raw_train_df.comment_text.apply(lambda x: clean(x))
print('train cleaned ...')

%time test_df['comment_text_cleaned'] = test_df.comment_text.apply(lambda x: clean(x))
print('test cleaned ...')

CPU times: user 8.24 s, sys: 45.4 ms, total: 8.29 s
Wall time: 8.29 s
train cleaned ...
CPU times: user 22.1 s, sys: 115 ms, total: 22.2 s
Wall time: 22.2 s
test cleaned ...


Feature engineering (adding columns that may be useful for modeling and analysis)

In [11]:
def add_cols(df, txt_col='text'):
    rows = []
    
    for index, row in df.iterrows():
        cols = OrderedDict({
            'token_count': 0.0,
            'named_entity_count': 0.0,
            'word_count': 0.0,
            'unique_word_count': 0.0,
            'unique_word_pct': 0.0,
            'upper_word_count': 0.0,
            'avg_word_len': 0.0,
            'char_count': 0.0,
            'unique_char_count': 0.0,
            'upper_char_count': 0.0,
            
            'stopwords_count': 0.0,
            'punctuation_count': 0.0,
            'punctuation_pct': 0.0,
            'symbol_count': 0.0,
            'number_count': 0.0,
            'alpha_count': 0.0,
            'noun_count': 0.0,
            'verb_count': 0.0,
            'adj_count': 0.0,
            'proper_name_count': 0.0,

            # named entity types
            'named_entity_count': 0.0,
            'named_entity_person_count': 0.0,
            'named_entity_norp_count': 0.0,
            'named_entity_facility_count': 0.0,
            'named_entity_org_count': 0.0,
            'named_entity_gpe_count': 0.0,
            'named_entity_non_gpe_loc_count': 0.0,
            'named_entity_product_count': 0.0,
            'named_entity_event_count': 0.0,
            'named_entity_woa_count': 0.0,
            'named_entity_lang_count': 0.0,
            'named_entity_date_count': 0.0,
            'named_entity_time_count': 0.0,
            'named_entity_money_count': 0.0,
            'named_entity_quantity_count': 0.0
        })
        
        # grab tokens, entities, and word tokens
        txt = row[txt_col]
        tokens = spacy_en(txt)
        ents = tokens.ents
        words = txt.split()
        
        # get counts
        cols['token_count'] = len(tokens)
        cols['ent_count'] = len(ents)
        cols['word_count'] = len(words)
        cols['unique_word_count'] = len(set(words))
        cols['upper_word_count'] = len([w for w in words if (w.isupper())])
        cols['avg_word_len'] = np.mean([len(w) for w in words])
        
        cols['char_count'] = len(txt)
        cols['unique_char_count'] = len(set(txt))
        cols['upper_char_count'] = len([c for c in txt if (c.isupper())])
        
        for t in tokens:
            if (t.is_punct): cols['punctuation_count'] += 1
            if (t.is_stop): cols['stopwords_count'] += 1
            if (t.pos_ == 'SYM'): cols['symbol_count'] += 1
            if (t.pos_ == 'NUM'): cols['number_count'] += 1
            if (t.is_alpha): cols['alpha_count'] += 1
                
            if (t.pos_ == 'NOUN') : cols['noun_count'] += 1
            if (t.pos_ == 'VERB') : cols['verb_count'] += 1
            if (t.pos_ == 'ADJ') : cols['adj_count'] += 1
            if (t.pos_ == 'PROPN') : cols['proper_name_count'] += 1
                
        for ent in ents:
            if (ent.label_ == 'PERSON'): cols['proper_name_count'] += 1
            if (ent.label_ == 'NORP'): cols['named_entity_norp_count'] += 1
            if (ent.label_ == 'FACILITY'): cols['named_entity_facility_count'] += 1
            if (ent.label_ == 'ORG'): cols['named_entity_org_count'] += 1
            if (ent.label_ == 'GPE'): cols['named_entity_gpe_count'] += 1
            if (ent.label_ == 'LOC'): cols['named_entity_non_gpe_loc_count'] += 1
            if (ent.label_ == 'PRODUCT'): cols['named_entity_product_count'] += 1
            if (ent.label_ == 'EVENT'): cols['named_entity_event_count'] += 1
            if (ent.label_ == 'WORK_OF_ART'): cols['named_entity_woa_count'] += 1
            if (ent.label_ == 'LANGUAGE'): cols['named_entity_lang_count'] += 1
            if (ent.label_ == 'DATE'): cols['named_entity_date_count'] += 1
            if (ent.label_ == 'TIME'): cols['named_entity_time_count'] += 1
            if (ent.label_ == 'MONEY'): cols['named_entity_money_count'] += 1
            if (ent.label_ == 'QUANTITY'): cols['named_entity_quantity_count'] += 1

        cols['unique_word_pct'] = (cols['unique_word_count'] + 1) / (cols['word_count'] + 1)
        cols['unique_char_pct'] = (cols['unique_char_count'] + 1) / (cols['char_count'] + 1)
        cols['punctuation_pct'] = (cols['punctuation_count'] + 1) / (cols['token_count'] + 1)
        
        rows.append(cols)
        
    return pd.DataFrame(rows, columns=cols.keys())

In [12]:
%time raw_train_df = pd.concat([raw_train_df, add_cols(raw_train_df, 'comment_text')], axis=1)
print('train feature engineering done ...')

%time test_df = pd.concat([test_df, add_cols(test_df, 'comment_text')], axis=1)
print('test feature engineering done ...')

CPU times: user 1h 50min 59s, sys: 3min 2s, total: 1h 54min 1s
Wall time: 59min 35s
train feature engineering done ...


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 1h 45min 57s, sys: 2min 46s, total: 1h 48min 44s
Wall time: 56min 55s
test feature engineering done ...


Let's see what we've done

In [11]:
display(raw_train_df.head(3))
display(test_df.head(3))

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none,comment_text_cleaned
0,22256635,"Nonsense? kiss off, geek. what I said is true. I'll have your account terminated.",1,0,0,0,0,0,0,"nonsense? kiss off, geek. what i said is true. i will have your account terminated."
1,27450690,"""\n\n Please do not vandalize pages, as you did with this edit to W. S. Merwin. If you continue to do so, you will be blocked from editing. """,0,0,0,0,0,0,1,""" please do not vandalize pages, as you did with this edit to w. s. merwin. if you continue to do so, you will be blocked from editing. """
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the """"points of interest"""" section you added because it seemed kind of spammy. I know you probably didn't mean to disobey the rules, but generally, a point of interest tends to be rather touristy, and quite irrelevant to an area culture. That's just my opinion, though.\n\nIf you want to reply, just put your reply here and add {{talkback|Jamiegraham08}} on my talkpage. """,0,0,0,0,0,0,1,""" """"points of interest"""" i removed the """"points of interest"""" section you added because it seemed kind of spammy. i know you probably did not mean to disobey the rules, but generally, a point of interest tends to be rather touristy, and quite irrelevant to an area culture. that is just my opinion, though. if you want to reply, just put your reply here and add {{talkback|jamiegraham08}} on my talkpage. """


,id,comment_text,comment_text_cleaned
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. SS500 .jpg)==,==orphaned non-free media (image:41cd1jboevl. ss500 .jpg)==
1,6102620,"::Kentuckiana is colloquial. Even though the area is often referred to as this, it (in my opinion) has never held the encyclopedic precision of ""Louisville metropolitian area"", which has a specific U.S. Census definition. Also, apparently Kentuckiana often refers to the local television viewing area, which isn't nearly contiguous with the official metro area. As you indicate, Kentuckiana seems to be more of a slang or marketing phenomena than anything we could pin down in encyclopedic terms here. That's why we see Wikipedia language like ""the Louisville metropolitan area, sometimes referred to as Kentuckiana"". That's my take on it. — •","::kentuckiana is colloquial. even though the area is often referred to as this, it (in my opinion) has never held the encyclopedic precision of ""louisville metropolitian area"", which has a specific u.s. census definition. also, apparently kentuckiana often refers to the local television viewing area, which is not nearly contiguous with the official metro area. as you indicate, kentuckiana seems to be more of a slang or marketing phenomena than anything we could pin down in encyclopedic terms here. that is why we see wikipedia language like ""the louisville metropolitan area, sometimes referred to as kentuckiana"". that is my take on it. — •"
2,14563293,"Hello fellow Wikipedians,\nI have just modified on [WIKI_LINK: Double Trouble (George Jones and Johnny Paycheck album)]. Please take a moment to review [EXTERNAL_LINK: my edit]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit for additional information. I made the following changes:\n*Added archive [EXTERNA_LINK: https://web.archive.org/web/20070403204132/http://www.epiccenter.com/] to [EXTERNA_LINK: http://www.epiccenter.com/]\nWhen you have finished reviewing my changes, please set the checked parameter below to true or failed to let others know (documentation at ).\nCheers.—","hello fellow wikipedians, i have just modified on [wiki_link: double trouble (george jones and johnny paycheck album)]. please take a moment to review [external_link: my edit]. if you have any questions, or need the bot to ignore the links, or the page altogether, please visit for additional information. i made the following changes: *added archive [externa_link: https://web.archive.org/web/ /http://www.epiccenter.com/] to [externa_link: http://www.epiccenter.com/] when you have finished reviewing my changes, please set the checked parameter below to true or failed to let others know (documentation at ). cheers.—"


Save preprocessed data

In [12]:
raw_train_df.to_csv(f'{PATH}/train_preproc.csv', index=None)
test_df.to_csv(f'{PATH}/test_preproc.csv', index=None)